In [1]:
# pip install transformers torchaudio sounddevice arabic-reshaper pydub -q

In [2]:
import json
import torch
import sounddevice as sd
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from difflib import SequenceMatcher

# Load the Whisper model and processor
processor = WhisperProcessor.from_pretrained("tarteel-ai/whisper-tiny-ar-quran")
model = WhisperForConditionalGeneration.from_pretrained("tarteel-ai/whisper-tiny-ar-quran")

C:\Users\youss\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Mapping of Surah names to JSON files
surah_files = {
    "Al-Fatiha (الفاتحة)": "surahs_json_files/001_al-fatiha.json",
    "Al-Baqarah (البقرة)": "surahs_json_files/002_al-baqarah.json",
    "Aal-E-Imran (آل عمران)": "surahs_json_files/003_aal-e-imran.json",
    "An-Nisa (النساء)": "surahs_json_files/004_an-nisa.json",
    "Al-Maidah (المائدة)": "surahs_json_files/005_al-maidah.json",
    "Al-An'am (الأنعام)": "surahs_json_files/006_al-anam.json",
    "Al-A'raf (الأعراف)": "surahs_json_files/007_al-a'raf.json",
    "Al-Anfal (الأنفال)": "surahs_json_files/008_al-anfal.json",
    "At-Tawbah (التوبة)": "surahs_json_files/009_at-tawbah.json",
    "Yunus (يونس)": "surahs_json_files/010_yunus.json",
    "Hud (هود)": "surahs_json_files/011_hud.json",
    "Yusuf (يوسف)": "surahs_json_files/012_yusuf.json",
    "Ar-Ra'd (الرعد)": "surahs_json_files/013_ar-rad.json",
    "Ibrahim (إبراهيم)": "surahs_json_files/014_ibrahim.json",
    "Al-Hijr (الحجر)": "surahs_json_files/015_al-hijr.json",
    "An-Nahl (النحل)": "surahs_json_files/016_an-nahl.json",
    "Al-Isra (الإسراء)": "surahs_json_files/017_al-isra.json",
    "Al-Kahf (الكهف)": "surahs_json_files/018_al-kahf.json",
    "Maryam (مريم)": "surahs_json_files/019_maryam.json",
    "Ta-Ha (طه)": "surahs_json_files/020_ta-ha.json",
    "Al-Anbiya (الأنبياء)": "surahs_json_files/021_al-anbiya.json",
    "Al-Hajj (الحج)": "surahs_json_files/022_al-hajj.json",
    "Al-Mu'minun (المؤمنون)": "surahs_json_files/023_al-mu'minun.json",
    "An-Nur (النور)": "surahs_json_files/024_an-nur.json",
    "Al-Furqan (الفرقان)": "surahs_json_files/025_al-furqan.json",
    "Ash-Shu'ara (الشعراء)": "surahs_json_files/026_ash-shu'ara.json",
    "An-Naml (النمل)": "surahs_json_files/027_an-naml.json",
    "Al-Qasas (القصص)": "surahs_json_files/028_al-qasas.json",
    "Al-Ankabut (العنكبوت)": "surahs_json_files/029_al-ankabut.json",
    "Ar-Rum (الروم)": "surahs_json_files/030_ar-rum.json",
    "Luqman (لقمان)": "surahs_json_files/031_luqman.json",
    "As-Sajdah (السجدة)": "surahs_json_files/032_as-sajdah.json",
    "Al-Ahzab (الأحزاب)": "surahs_json_files/033_al-ahzab.json",
    "Saba (سبأ)": "surahs_json_files/034_saba.json",
    "Fatir (فاطر)": "surahs_json_files/035_fatir.json",
    "Ya-Sin (يس)": "surahs_json_files/036_ya-sin.json",
    "As-Saffat (الصافات)": "surahs_json_files/037_as-saffat.json",
    "Sad (ص)": "surahs_json_files/038_sad.json",
    "Az-Zumar (الزمر)": "surahs_json_files/039_az-zumar.json",
    "Ghafir (غافر)": "surahs_json_files/040_ghafir.json",
    "Fussilat (فصلت)": "surahs_json_files/041_fussilat.json",
    "Ash-Shura (الشورى)": "surahs_json_files/042_ash-shura.json",
    "Az-Zukhruf (الزخرف)": "surahs_json_files/043_az-zukhruf.json",
    "Ad-Dukhkhan (الدخان)": "surahs_json_files/044_ad-dukhkhan.json",
    "Al-Jathiya (الجاثية)": "surahs_json_files/045_al-jathiya.json",
    "Al-Ahqaf (الأحقاف)": "surahs_json_files/046_al-ahqaf.json",
    "Muhammad (محمد)": "surahs_json_files/047_muhammad.json",
    "Al-Fath (الفتح)": "surahs_json_files/048_al-fath.json",
    "Al-Hujurat (الحجرات)": "surahs_json_files/049_al-hujurat.json",
    "Qaf (ق)": "surahs_json_files/050_qaf.json",
    "Adh-Dhariyat (الذاريات)": "surahs_json_files/051_adh-dhariyat.json",
    "At-Tur (الطور)": "surahs_json_files/052_at-tur.json",
    "An-Najm (النجم)": "surahs_json_files/053_an-najm.json",
    "Al-Qamar (القمر)": "surahs_json_files/054_al-qamar.json",
    "Ar-Rahman (الرحمن)": "surahs_json_files/055_ar-rahman.json",
    "Al-Waqi'ah (الواقعة)": "surahs_json_files/056_al-waqi'ah.json",
    "Al-Hadid (الحديد)": "surahs_json_files/057_al-hadid.json",
    "Al-Mujadila (المجادلة)": "surahs_json_files/058_al-mujadila.json",
    "Al-Hashr (الحشر)": "surahs_json_files/059_al-hashr.json",
    "Al-Mumtahina (الممتحنة)": "surahs_json_files/060_al-mumtahina.json",
    "As-Saff (الصف)": "surahs_json_files/061_as-saff.json",
    "Al-Jumu'ah (الجمعة)": "surahs_json_files/062_al-jumu'ah.json",
    "Al-Munafiqoon (المنافقون)": "surahs_json_files/063_al-munafiqoon.json",
    "At-Taghabun (التغابن)": "surahs_json_files/064_at-taghabun.json",
    "At-Talaq (الطلاق)": "surahs_json_files/065_at-talaq.json",
    "At-Tahrim (التحريم)": "surahs_json_files/066_at-tahrim.json",
    "Al-Mulk (الملك)": "surahs_json_files/067_al-mulk.json",
    "Al-Qalam (القلم)": "surahs_json_files/068_al-qalam.json",
    "Al-Haqqah (الحاقة)": "surahs_json_files/069_al-haqqah.json",
    "Al-Ma'arij (المعارج)": "surahs_json_files/070_al-ma'arij.json",
    "Nooh (نوح)": "surahs_json_files/071_nooh.json",
    "Al-Jinn (الجن)": "surahs_json_files/072_al-jinn.json",
    "Al-Muzzammil (المزمل)": "surahs_json_files/073_al-muzzammil.json",
    "Al-Muddathir (المدثر)": "surahs_json_files/074_al-muddathir.json",
    "Al-Qiyamah (القيامة)": "surahs_json_files/075_al-qiyamah.json",
    "Al-Insan (الإنسان)": "surahs_json_files/076_al-insan.json",
    "Al-Mursalat (المرسلات)": "surahs_json_files/077_al-mursalat.json",
    "An-Naba (النبأ)": "surahs_json_files/078_an-naba.json",
    "An-Nazi'at (النازعات)": "surahs_json_files/079_an-nazi'at.json",
    "Abasa (عبس)": "surahs_json_files/080_abasa.json",
    "At-Takwir (التكوير)": "surahs_json_files/081_at-takwir.json",
    "Al-Infitar (الإنفطار)": "surahs_json_files/082_al-infitar.json",
    "Al-Mutaffifin (المطففين)": "surahs_json_files/083_al-mutaffifin.json",
    "Al-Inshiqaq (الإنشقاق)": "surahs_json_files/084_al-inshiqaq.json",
    "Al-Buruj (البروج)": "surahs_json_files/085_al-buruj.json",
    "At-Tariq (الطارق)": "surahs_json_files/086_at-tariq.json",
    "Al-A'la (الأعلى)": "surahs_json_files/087_al-a'la.json",
    "Al-Ghashiyah (الغاشية)": "surahs_json_files/088_al-ghashiyah.json",
    "Al-Fajr (الفجر)": "surahs_json_files/089_al-fajr.json",
    "Al-Balad (البلد)": "surahs_json_files/090_al-balad.json",
    "Ash-Shams (الشمس)": "surahs_json_files/091_ash-shams.json",
    "Al-Lail (الليل)": "surahs_json_files/092_al-layl.json",
    "Ad-Duha (الضحى)": "surahs_json_files/093_ad-duha.json",
    "Ash-Sharh (الشرح)": "surahs_json_files/094_ash-sharh.json",
    "At-Tin (التين)": "surahs_json_files/095_at-tin.json",
    "Al-Alaq (العلق)": "surahs_json_files/096_al-'alaq.json",
    "Al-Qadr (القدر)": "surahs_json_files/097_al-qadr.json",
    "Al-Bayyina (البينة)": "surahs_json_files/098_al-bayyina.json",
    "Az-Zalzalah (الزلزلة)": "surahs_json_files/099_az-zalzalah.json",
    "Al-Adiyat (العاديات)": "surahs_json_files/100_al-adiyat.json",
    "Al-Qari'ah (القارعة)": "surahs_json_files/101_al-qari'ah.json",
    "At-Takathur (التكاثر)": "surahs_json_files/102_at-takathur.json",
    "Al-Asr (العصر)": "surahs_json_files/103_al-asr.json",
    "Al-Humazah (الهمزة)": "surahs_json_files/104_al-humazah.json",
    "Al-Fil (الفيل)": "surahs_json_files/105_al-fil.json",
    "Quraish (قريش)": "surahs_json_files/106_quraish.json",
    "Al-Ma'un (الماعون)": "surahs_json_files/107_al-ma'un.json",
    "Al-Kawthar (الكوثر)": "surahs_json_files/108_al-kawthar.json",
    "Al-Kafirun (الكافرون)": "surahs_json_files/109_al-kafirun.json",
    "An-Nasr (النصر)": "surahs_json_files/110_an-nasr.json",
    "Al-Masad (المسد)": "surahs_json_files/111_al-masad.json",
    "Al-Ikhlas (الإخلاص)": "surahs_json_files/112_al-ikhlas.json",
    "Al-Falaq (الفلق)": "surahs_json_files/113_al-falaq.json",
    "An-Nas (الناس)": "surahs_json_files/114_an-nas.json",
}

In [5]:
# Function to load the surahs with their verses
def load_all_surahs():
    quran_text_arabic = {}
    
    for surah_name, json_file in surah_files.items():
        with open(json_file, "r", encoding="utf-8") as file:
            quran_data = json.load(file)
            
            # Check if the structure is as expected
            if 'ayahs' in quran_data and isinstance(quran_data['ayahs'], list):
                # Extract verses
                verses = [ayah['text'] for ayah in quran_data['ayahs']]
                quran_text_arabic[surah_name] = verses
            else:
                print(f"Unexpected structure in {json_file}")
    
    return quran_text_arabic

In [6]:
quran_text_arabic = load_all_surahs()

In [7]:
# Function to record audio from the microphone
def record_audio(duration=5, fs=16000):
    print("Recording...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()  # Wait until the recording is finished
    print("Recording finished")
    return torch.tensor(audio).T

# Function to load audio from a file
def load_audio_file(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    return waveform, sample_rate

# Function to preprocess and transcribe audio
def transcribe_audio(audio, sample_rate=16000):
    # Preprocess the audio
    inputs = processor(audio.squeeze().numpy(), sampling_rate=sample_rate, return_tensors="pt")

    # Extract input features
    input_features = inputs.input_features
    
    # Generate the transcription
    print("Generating transcription with Whisper")
    predicted_ids = model.generate(input_features)
    
    # Decode the transcription
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    print(f"Transcription of the audio: {transcription}")
    return transcription
    
# Function to get transcription from either microphone or file
def speech_to_text_arabic(source, source_type='mic', duration=5, file_path=None):
    if source_type == 'mic':
        # Record audio from the microphone
        speech_array = record_audio(duration=duration, fs=16000)
        return transcribe_audio(speech_array)
    elif source_type == 'file' and file_path:
        # Load and process the audio file
        audio, sample_rate = load_audio_file(file_path)
        return transcribe_audio(audio, sample_rate)
    else:
        raise ValueError("Invalid source_type or missing file_path for file source")

# Function to calculate similarity between two strings
def calculate_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Function to find the best matching Ayah from the transcription
def find_ayah_arabic(transcription, threshold=0.6):
    print("Searching for the matching Ayah in Quran text with similarity check")
    best_match = None
    best_score = 0.0
    
    for surah, ayahs in quran_text_arabic.items():
        for ayah_num, ayah in enumerate(ayahs, start=1):
            similarity_score = calculate_similarity(transcription, ayah)
            if similarity_score > best_score:
                best_score = similarity_score
                best_match = (surah, ayah_num)
            
            if best_score == 1.0:  # Exact match found
                print(f"Exact match found: Surah {surah}, Ayah {ayah_num}")
                return surah, ayah_num

    if best_score >= threshold:
        surah, ayah_num = best_match
        print(f"Best match (similarity {best_score:.2f}): Surah {surah}, Ayah {ayah_num}")
        return surah, ayah_num
    
    print("No sufficiently close match found")
    return None, None

# Main function to integrate everything
def identify_quranic_ayah(source, source_type='mic', duration=5, file_path=None):
    # Get transcription from the chosen source
    transcription = speech_to_text_arabic(source, source_type=source_type, duration=duration, file_path=file_path)
    
    # Find the closest matching Ayah
    surah, ayah_num = find_ayah_arabic(transcription)
    
    if surah and ayah_num:
        return f"الآية من سورة {surah}، رقم الآية {ayah_num}."
    else:
        return "لم يتم التعرف على الآية."

In [8]:
# Example usage
result = identify_quranic_ayah(source=None, source_type='mic', duration=10)
print(result)

Recording...
Recording finished
Generating transcription with Whisper


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcription of the audio: فَالطَّفِّرِ
Searching for the matching Ayah in Quran text with similarity check
Best match (similarity 0.64): Surah At-Tur (الطور), Ayah 1
الآية من سورة At-Tur (الطور)، رقم الآية 1.


In [ ]:
# Example usage
result = identify_quranic_ayah(source=None, source_type='mic', duration=10)
print(result)